In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
#from datasets import load_dataset
import pandas as pd
import warnings


warnings.filterwarnings('ignore')


In [ ]:

# Load pre-trained model and tokenizer
model_name = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=3)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
!pip install datasets


from datasets import load_dataset

dataset = load_dataset("text", data_files={"train": "labeled_dataset.conll"})
print(dataset)

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 8933
    })
})


In [20]:
from google.colab import files
uploaded = files.upload()

Saving labeled_dataset.conll to labeled_dataset (1).conll


In [21]:
# Display the uploaded files to ensure the file is there
for filename in uploaded.keys():
    print(f'Uploaded file: {filename}')

Uploaded file: labeled_dataset (1).conll


In [22]:

import pandas as pd

# Get the filename from the uploaded dictionary
filename = list(uploaded.keys())[0]
print(f'Loading file: {filename}')

# Initialize lists to hold tokens and labels
tokens = []
labels = []

# Read the file line by line
with open(filename, 'r') as file:
    for line in file:
        line = line.strip()
        if line:  # Skip empty lines
            parts = line.split(maxsplit=1)  # Split by the first space only
            if len(parts) == 2:  # Ensure there are exactly 2 parts
                token, label = parts
                tokens.append(token)
                labels.append(label)
            else:
                print(f"Skipping line: {line}")  # Debugging output for lines that don't match the format

# Create a DataFrame
df = pd.DataFrame({'Token': tokens, 'Label': labels})

# Display the first few rows of the DataFrame to ensure it was loaded correctly
print(df.head(10))


Loading file: labeled_dataset (1).conll
    Token    Label
0    በበርዎ  B-PRICE
1      ስር        O
2     አቧራ        O
3      እና        O
4    ነፍሳት        O
5  እንዳይገባ        O
6  የሚከላከል        O
7   አንደኛው        O
8    ወፍራሙ        O
9     ዋጋ፦  B-PRICE


In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

# Load the model
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(label_mapping))

# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",  # Change to eval_strategy
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Fine-tune the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=dataset,
)

trainer.train()

# Evaluate the model
results = trainer.evaluate()
print(results)

# Save the model
model.save_pretrained('./fine-tuned-ner-model')
tokenizer.save_pretrained('./fine-tuned-ner-model')